<h1 style="color:gold; background-color:black; padding:20px">TRADUCTOR QUECHUA - ESPAÑOL / ESPAÑOL - QUECHUA</h1>

<h1>Importar librerias a utilizar</h1>

In [1]:
import json
import numpy as np
import os
import pandas as pd

In [2]:
import pydot
import graphviz

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

<h1>Definir funciones auxiliares</h1>

<h3 style="color:crimson">Imprime barra de progreso</h3>

In [4]:
def imprimir_barra_progreso_v2(progreso, total):
    # https://www.youtube.com/watch?v=x1eaT88vJUA
    
    p = int(100 * (progreso + 1)/total)

    # Alt+291: █
    barra = '█'*p + '-'*(100-p) 

    print("\r|{}| {}%".format(barra, p), end='\r')

<h1>Importar datos</h1>

<p>Los archivos contienen palabras/oraciones paralelas
<br>
La primera columna está en español y la segunda en quechua</p>

In [5]:
# Crear un dataframe vació que almacenará las traducciones (translations)
df_trans = pd.DataFrame(columns=["español","quechua"])
df_trans

,español,quechua


<p>Se utilizó un tab ("\t") como separador de columnas para evitar confusiones con las comas propias de las oraciones</p>

<h3 style="color:crimson">Importa DataFrames</h3>

In [6]:
sub_paths = ["./Datos/palabras/",
             "./Datos/Grupos/",
             "./Datos/libros/"]

for sub_path in sub_paths:
    arr_category = os.listdir(sub_path)

    for item in arr_category:
        if item[-4:] == ".csv":
            file = "{}{}".format(sub_path,item)
            df_temp = pd.read_csv(file, encoding="utf-8", sep="\t")
            
            df_trans = pd.concat([df_trans, df_temp]
                                 , ignore_index = True)

In [7]:
df_trans

,español,quechua
0,ácido,ácido nisqa
1,agradable,munasqa
2,agrícola,chakra llamk’aymanta
3,algún,wakin
4,amable,kuyakuq
...,...,...
17576,que aumenten sus fatigas tu tesoro;,qhapaq kayniyki llank’ayninkuta yapachun;
17577,y cambia horas de espuma por divinas.,hinaspa horas de espuma cambiay divinopaq.
17578,"Sé rica adentro, en vez de serlo afuera.","Hawamanta qhapaq kaymantaqa, ukhupi qhapaq kay."
17579,"Devora tú a la Muerte y no la nutras,","Wañuytaqa mikhunkichis, manataq mikhuchinkich..."


<h3 style="color:crimson">Eliminar frases muy largas</h3>
<p>Antes de agregar esta línea, se generaba el siguiente error al separar espacio en memoria para los arreglos de numpy</p>
<p>Unable to allocate 152. GiB for an array with shape (18435, 18240, 121) and data type float32</p>

In [8]:
for index, registro in df_trans.iterrows():
    txt_es = registro[0]
    txt_qu = registro[1]
    
    if (len(txt_es) > 250) or (len(txt_qu) > 250):
        #print("registro eliminado")
        df_trans = df_trans.drop(index)

registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eli

registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eliminado
registro eli

In [9]:
df_trans

,español,quechua
0,ácido,ácido nisqa
1,agradable,munasqa
2,agrícola,chakra llamk’aymanta
3,algún,wakin
4,amable,kuyakuq
...,...,...
17576,que aumenten sus fatigas tu tesoro;,qhapaq kayniyki llank’ayninkuta yapachun;
17577,y cambia horas de espuma por divinas.,hinaspa horas de espuma cambiay divinopaq.
17578,"Sé rica adentro, en vez de serlo afuera.","Hawamanta qhapaq kaymantaqa, ukhupi qhapaq kay."
17579,"Devora tú a la Muerte y no la nutras,","Wañuytaqa mikhunkichis, manataq mikhuchinkich..."


<h1>Configuración</h1>

In [10]:
batch_size = 64 # tamño de los lotes para entrenamiento
epochs = 100 # Número de epochs
latent_dim = 256 # dimensión del espacio latente para el encoder
num_samples = 10000

<h1>Preparar los datos</h1>

In [11]:
# Vectoriza los datos
i=0
targe_text= ''

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [12]:
for index, registro in df_trans.iterrows():    
    
    input_text = registro[0]
    target_text = registro[1]
    
    
    if (index<10):
        print("{} \t I: {} \t T: {}"
              .format(index, input_text, targe_text))        

    
    # Usaremos "tab" como el  caracter de inicio (start sequence)
    # para los targets, y "\n" como el caracter de fin de secuencia "end sequence"
    target_text = "\t" + target_text + "\n"
    # sube las líneas a  las listas
    input_texts.append(input_text)
    target_texts.append(target_text)
  
    # completa los conjuntos de caracteres si es necesario
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
    i=i+1

0 	 I: ácido 	 T: 
1 	 I: agradable 	 T: 
2 	 I: agrícola 	 T: 
3 	 I: algún 	 T: 
4 	 I: amable 	 T: 
5 	 I: amargo 	 T: 
6 	 I: ambos 	 T: 
7 	 I: ancho 	 T: 
8 	 I: aquel 	 T: 
9 	 I: aquellas 	 T: 


In [13]:

# Convierte los dos conjuntos de caracteres
# en dos listas ordenadas
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))  
# calcule el número de tokens (caracteres) en ambos lados
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
# calcula la máxima longitud de las secuencias en cada lado
max_encoder_seq_length = max([len(text) for text in input_texts])
max_decoder_seq_length = max([len(text) for text in target_texts])

In [14]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)
print("preparando datos...")

Number of samples: 16878
Number of unique input tokens: 117
Number of unique output tokens: 109
Max sequence length for inputs: 250
Max sequence length for outputs: 252
preparando datos...


In [15]:
# crea diccionarios de tokens
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

# crea los tensores 1-hot para el encoder y el decoder
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

In [16]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

print ("\n....")
print("datos preparados")


....
datos preparados


<h1>Construir el modelo</h1>

<h3 style="color:crimson">Encoder</h3>

In [17]:
# define una secuencia de entrada y la procesa
encoder_inputs = Input(shape = (None, num_encoder_tokens))

# capa recurrente del encoder
encoder = LSTM(latent_dim, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# Descartamos las salidas (encoder_outputs)
# solamente se conserva las memoria de  corto (state_h) y 
# largo plazo(state_c)
encoder_states = [state_h, state_c]

<h3 style="color:crimson">Decoder</h3>

In [18]:
# Configuramos el decoder, usando 'encoder_states' como estado inicial
decoder_inputs = Input(shape= (None, num_decoder_tokens))

# capa recurrente del decoder
# Configuramos nuestro decodificador para devolver secuencias de salida completas,
# y también para devolver estados internos. No usamos los
# estados retornados en el modelo de entrenamiento, pero los usaremos en inferencia.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _,_ = decoder_lstm(decoder_inputs,initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

<h3 style="color:crimson">Modelo Completo</h3>

In [19]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [20]:
model.summary()
plot_model(model, to_file='../Imagenes/s2s.png', 
           show_shapes=True)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 117)]  0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 109)]  0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 256),        382976      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                              

<h1>Entrenar el modelo</h1>

In [21]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [22]:
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
history = model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

Epoch 1/100
211/211 [==============================] - 21s 78ms/step - loss: 1.2252 - accuracy: 0.7170 - val_loss: 0.9712 - val_accuracy: 0.7256
Epoch 2/100
211/211 [==============================] - 15s 69ms/step - loss: 0.8358 - accuracy: 0.7698 - val_loss: 0.7738 - val_accuracy: 0.7800
Epoch 3/100
211/211 [==============================] - 15s 71ms/step - loss: 0.7133 - accuracy: 0.7947 - val_loss: 0.7119 - val_accuracy: 0.7931
Epoch 4/100
211/211 [==============================] - 15s 71ms/step - loss: 0.6671 - accuracy: 0.8051 - val_loss: 0.6763 - val_accuracy: 0.8021
Epoch 5/100
211/211 [==============================] - 15s 72ms/step - loss: 0.6342 - accuracy: 0.8149 - val_loss: 0.6458 - val_accuracy: 0.8125
Epoch 6/100
211/211 [==============================] - 15s 70ms/step - loss: 0.6084 - accuracy: 0.8227 - val_loss: 0.6245 - val_accuracy: 0.8181
Epoch 7/100
211/211 [==============================] - 14s 69ms/step - loss: 0.5863 - accuracy: 0.8282 - val_loss: 0.6063 - val_ac

211/211 [==============================] - 14s 69ms/step - loss: 0.3238 - accuracy: 0.9000 - val_loss: 0.4272 - val_accuracy: 0.8723
Epoch 58/100
211/211 [==============================] - 14s 69ms/step - loss: 0.3221 - accuracy: 0.9005 - val_loss: 0.4267 - val_accuracy: 0.8723
Epoch 59/100
211/211 [==============================] - 14s 68ms/step - loss: 0.3206 - accuracy: 0.9009 - val_loss: 0.4273 - val_accuracy: 0.8726
Epoch 60/100
211/211 [==============================] - 15s 69ms/step - loss: 0.3191 - accuracy: 0.9014 - val_loss: 0.4281 - val_accuracy: 0.8722
Epoch 61/100
211/211 [==============================] - 14s 69ms/step - loss: 0.3178 - accuracy: 0.9018 - val_loss: 0.4272 - val_accuracy: 0.8728
Epoch 62/100
211/211 [==============================] - 14s 69ms/step - loss: 0.3164 - accuracy: 0.9022 - val_loss: 0.4296 - val_accuracy: 0.8722
Epoch 63/100
211/211 [==============================] - 14s 68ms/step - loss: 0.3151 - accuracy: 0.9025 - val_loss: 0.4291 - val_accuracy

<h1>Guardar el modelo</h1>

In [25]:
import json

In [26]:
with open("./Modelos/s2q/input_token_index.txt", "w") as f:
    json.dump(input_token_index, f)

In [27]:
with open("./Modelos/s2q/target_token_index.txt", "w") as f:
    json.dump(target_token_index, f)

In [28]:
with open("./Modelos/s2q/encoder_input_data.npy", "wb") as f:
    np.save(f, encoder_input_data)

In [29]:
print("num_decoder_tokens: {}".format(num_decoder_tokens))
print("max_decoder_seq_length: {}".format(max_decoder_seq_length))

num_decoder_tokens: 109
max_decoder_seq_length: 252


In [30]:
# s2q = Spanish to Quechua
model.save("./Modelos/s2q/spanish_to_quechua")

INFO:tensorflow:Assets written to: ./Modelos/s2q/spanish_to_quechua\assets


INFO:tensorflow:Assets written to: ./Modelos/s2q/spanish_to_quechua\assets
